In [1]:
# MS20_Task5_SP_Client_Py3
import socket
import select
import _thread
import time
import numpy as np
from scipy.io import wavfile
from IPython import display as Dp
import PIL as pil
from matplotlib import pyplot as plt
%matplotlib inline

# Function to convert a string ("ABCDE") to a bitString (BS= 01000001...) 
def string_to_bitstring(Text):
    byteArr = bytes(Text,"latin-1")
    BA = np.unpackbits(bytearray(byteArr,'latin-1'))
    BS = ""
    for n in BA: BS += str(n)
    return BS

# Function to convert a bitstring BS (0100000 ...) to a text string ("ABCDE")
def bitstring_to_string(BS):
    BBA = bytearray(BS, "latin-1")  # bit-bytearray
    BA = np.uint8(BBA) - 48
    mySt = np.packbits(BA)
    mySt = mySt.tobytes()
    myString = mySt.decode('latin1')    # (use latin1 instead of utf-8)
    return myString

# Function to convert np.int16 array (NA) to bitstring (BS)
def numpyint16array_to_bitstring(NA):
    BS=np.unpackbits(bytearray(NA.byteswap()))
    BS=BS.astype(str)                 
    return "".join(BS)

# Function to convert a bitstring to an np.int16 array.
def bitstring_to_numpyint16array(BS):
    print("length of BS: ",len(BS))
    # Convert bit-string to bit-bytearray:
    BBA = bytearray(BS, "latin-1")  # bit-bytearray
    BA=np.uint8(BBA) - 48    # Convert to bitarray
    NA=np.packbits(BA)
    NA=NA.tobytes()
    NA=np.frombuffer(NA, dtype=np.int16)
    NA=NA.byteswap()
    return NA

# Function to convert np.uint8 array (NA) to bitstring (BS)
def uint8array_to_bitstring(NA):
    BS=np.unpackbits(bytearray(NA) )
    BS=BS.astype(str)                 
    return "".join(BS)

# Function to convert bitstring (BS) to np.uint8 array (NA)
def bitstring_to_numpyuint8array(BS):
    # Convert bit-string to bit-bytearray:
    BBA = bytearray(BS, "latin-1")  # bit-bytearray
    # Convert BBA to BA:
    BA = np.uint8(BBA) - 48
    NA=np.packbits(BA)
    NA=NA.tobytes()
    NA=np.frombuffer(NA, dtype=np.uint8)
    return NA

def send_to_server(msg):
    if len(msg) > 1024: print("send-message length = ", len(msg) )
    S.sendall(bytes(msg+"\n", "utf-8"))    # Send adds newline at end of msg
    return  # (Can be omitted)  
 
def receive_from_server(wait):
    # Receives long messages and time-out (returning zero message) after wait seconds
    Svr_msg = b""  ;  MTU =8192  # Note Svr_msg starts off as a bytes object
    waiting = select.select([S],[],[],wait) # Waits for wait seconds to see if there is msg on Svr 
    if waiting[0]:
        while 1:
            Svr_msg += S.recv(MTU)      # receive (MTU = buffer size)
            if Svr_msg[-1] == 10 : break       # 10 is code for newline
        # end of while clause
    Svr_msg = Svr_msg.decode("utf-8") # Convert to normal string 
    # end of if clause
    return Svr_msg.rstrip("\n")  #  Receiver removes newline on end of Svr_msg
    
def receive_from_keyboard():
    KB_msg = input("Enter a command: ")
    return KB_msg

def bschoose(KB_msg):
    msgs = KB_msg.split(" ",2)
    if len(msgs)<2:
        print("Error in choose: not enough arguments")
        msgtoS=""
        return
    towhom = msgs[1] ; what=msgs[2]
    if what=="T":
        if len(msgs)==4:  BS=msgs[3]   # bit-stream entered literally
        else:
            IFO = open("message.txt","rt")  # Read message from text-file
            Text = IFO.read()
            IFO.close
            BS = string_to_bitstring(Text)
        msgtoS = "BSMSG "+towhom+" T "+BS
        return msgtoS
    if what=="S":
        # Read in a wav file from working directory
        (Fs, speech) = wavfile.read("NarrobandSpeech8k.wav")
        print("Orig speech:")
        Dp.display(Dp.Audio(speech,rate=Fs))
        BS = numpyint16array_to_bitstring(speech)  #Testing
        msgtoS = "BSMSG "+towhom+" S "+BS
        return msgtoS    
    if what == "I":
        print( "Read in an image from working dir")
        chooseImage = "smallpeppers.bmp"        # 256 x 256 x 3
        img = pil.Image.open(chooseImage)
        img_array = np.array(img)  
        (Rows, Cols, Colours) = img_array.shape
        print("Rows, Cols, Colours:",Rows, Cols, Colours)
        plt.imshow(img_array)
        plt.show()
        time.sleep(0.1)
        # R, G, B = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
        Pic = np.reshape(img_array, (img_array.size))  # size gives total number of elements
        msgtoS = "BSMSG "+towhom+" I "+uint8array_to_bitstring(Pic)
        return msgtoS   
    if what == "J":
        chooseImage = "peppersjpg.jpg"        
        img = pil.Image.open(chooseImage)
        img_array = np.array(img)  
        plt.imshow(img_array)
        plt.show()
        print( "Read in the file again, now as data from working dir")
        IFO = open("peppersjpg.jpg","rb")
        x = IFO.read()
        NA = np.uint8(list(x))
        msgtoS = "BSMSG "+towhom+" J "+uint8array_to_bitstring(NA)
        IFO.close
        return msgtoS        
    if what == "D":
        chooseFile = ""   # Make this a file in working directory       
        if chooseFile == "": 
            print("No file specified")
            return   
        try:
            IFO = open(chooseFile,"rb")
        except: print("Error: file does not exist")
        x = IFO.read()
        NA = np.uint8(list(x))
        msgtoS = "BSMSG "+towhom+" D "+uint8array_to_bitstring(NA)
        IFO.close()
        return msgtoS
    else: 
        msgtoS = "Error in BSMSG message" ;  print(msgtoS)
    return
    
def bslisten(msgfromS):
    msgs = msgfromS.split(" ",3)
    fromwhom = msgs[1] ; what = msgs[2] 
    BS = msgs[3].rstrip("\n")   # Remove trailing endline (no need as receiver does it)
    if what=="T":
        Text = bitstring_to_string(BS)
        print("received text: ",Text)
        return
    if what == "S":
        speech = bitstring_to_numpyint16array(BS)
        print("Received speech:")
        Dp.display(Dp.Audio(speech,rate=8000))
        return
    if what=="I":
        Pic = bitstring_to_numpyuint8array(BS)
        img_array = np.reshape(Pic,(256,256,3))
        print("Received image:")
        plt.imshow(img_array)
        plt.show()
        return 
    if what == "J":
        NA = bitstring_to_numpyuint8array(BS)
        OFO = open("outfile.jpg","wb")
        y = bytes(NA)
        OFO.write(y)
        OFO.close
        try:
            img = pil.Image.open("outfile.jpg")
        except: 
            print("Image could not be opened")
            return
        img_array = np.array(img)
        print("Received image")
        try:
            plt.imshow(img_array)
        except:
            print("Image could not be displayed")
            return
        plt.show()
        return        
    if what == "D":
        if  BS =="": 
            print("Nothing rceived")
            return          
        NA = bitstring_to_numpyuint8array(BS)
        OFO = open("outfile.dat","wb")
        y = bytes(NA)
        OFO.write(y)
        OFO.close()
        return                               
    print("what is not T,S, I, J  or D in mylisten")
    # All other messages are ignored by mylisten
    return # (I know its not needed)

def lschoose(KB_msg):
    msgs = KB_msg.split(" ",2)
    if len(msgs)<2:
        print("Error in choose: not enough arguments")
        msgtoS=""
        return
    towhom = msgs[1] ; what=msgs[2]
    if what=="T":
        if len(msgs)==4:  BS=msgs[3]   # bit-stream entered literally
        else:
            IFO = open("message.txt","rt")  # Read message from text-file
            Text = IFO.read()
            IFO.close
        msgtoS = "LSMSG "+towhom+" T "+Text
        return msgtoS
    
    if what=="S":
        # Read in a wav file from working directory
        (Fs, speech) = wavfile.read("NarrobandSpeech8k.wav")
        print("Orig speech:")
        Dp.display(Dp.Audio(speech,rate=Fs))
        mybytes = speech.tobytes()
        SS = mybytes.decode("latin-1")               
        msgtoS = "LSMSG "+towhom+" S "+SS
        return msgtoS
    
    if what == "I":
        print( "Read in an image from working dir")
        chooseImage = "smallpeppers.bmp"        # 256 x 256 x 3
        img = pil.Image.open(chooseImage)
        img_array = np.array(img)  
        (Rows, Cols, Colours) = img_array.shape
        print("Rows, Cols, Colours:",Rows, Cols, Colours)
        plt.imshow(img_array)
        plt.show()
        time.sleep(0.1)
        # R, G, B = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
        Pic = np.reshape(img_array, (img_array.size))  # size gives total number of elements
        mybytes = Pic.tobytes()
        
        print("mybytes: ",mybytes[-8:])
        
        SS = mybytes.decode("latin-1")               
        msgtoS = "LSMSG "+towhom+" I "+SS
        return msgtoS
    
    if what == "J":
        chooseImage = "peppersjpg.jpg"        
        img = pil.Image.open(chooseImage)
        img_array = np.array(img)  
        plt.imshow(img_array)
        plt.show()
        print( "Read in the file again, now as data from working dir")
        IFO = open("peppersjpg.jpg","rb")
        x = IFO.read()
        NA = np.uint8(list(x))
        mybytes = NA.tobytes()
        SS = mybytes.decode("latin-1")
        msgtoS = "LSMSG "+towhom+" J "+SS
        IFO.close()
        return msgtoS
        
    if what == "D":
        chooseFile = ""   # Make this a file in working directory       
        if chooseFile == "": 
            print("No file specified")
            return   
        try:
            IFO = open(chooseFile,"rb")
        except: print("Error: file does not exist")
        x = IFO.read()
        NA = np.uint8(list(x))
        mybytes = NA.tobytes()
        SS = mybytes.decode("latin-1")
        msgtoS = "LSMSG "+towhom+" D "+SS              
        IFO.close()
        return msgtoS
    else: 
        msgtoS = "Error in LSMSG message" ;  print(msgtoS)
    return
    
def lslisten(msgfromS):
    msgs = msgfromS.split(" ",3)
    fromwhom = msgs[1] ; what = msgs[2] 
    # SS=msgs[3].rstrip() 
    SS=msgs[3].rstrip("\n")  # Prob not needed
    if what=="T":
        Text = SS
        print("received text: ",Text)
        return
    if what == "S":
        mybytes = SS.encode("latin-1")
        speech = np.frombuffer(mybytes, dtype=np.int16)
        print("Received speech:")
        Dp.display(Dp.Audio(speech,rate=8000))
        return
    if what=="I":
        mybytes = SS.encode("latin-1")
        print("mybytes: ",mybytes[-8:])
        Pic = np.frombuffer(mybytes, dtype=np.uint8)
        try:
            img_array = np.reshape(Pic,(256,256,3))
            print("Received image:")
            plt.imshow(img_array)
            plt.show()
        except: print("Image cannot be opened")
        return 
    if what == "J":
        mybytes = SS.encode("latin-1")
        NA = np.frombuffer(mybytes, dtype=np.uint8)
        OFO = open("outfile.jpg","wb")
        y = bytes(NA)
        OFO.write(y)
        OFO.close
        try:
            img = pil.Image.open("outfile.jpg")
        except: 
            print("Image could not be opened")
            return
        try:
            img_array = np.array(img)
            print("Received image")
            plt.imshow(img_array)
            plt.show()
        except:
            print("Image could not be displayed") 
        return        
    if what == "D":
        if  SS =="": 
            print("Nothing rceived")
            return          
        mybytes = SS.encode("latin-1")
        NA = np.frombuffer(mybytes, dtype=np.uint8)
        OFO = open("outfile.dat","wb")
        y = bytes(NA)
        OFO.write(y)
        OFO.close()
        return                               
    print("what is not T,S, I, J  or D in lslisten")
    # All other messages are ignored by mylisten
    return # (I know its not needed)
                                                              
def to_Print(msgtoS):
    L=len(msgtoS)
    if (L < 200): print("To Svr: ",msgtoS)  
    else:
        msg = msgtoS[0:200]+"..etc.."+msgtoS[L-8:L]
        print("To Svr: ",msg,"(length=",L,")")
        
def from_Print(msgfromS):
    L=len(msgfromS)
    if (L < 200): print("From Svr: ",msgfromS)  
    else:
        msg = msgfromS[0:200]+"..etc.."+msgfromS[L-8:L]
        print("From Svr: ",msg," (length=",L,")")        
        
S = socket.socket(socket.AF_INET,socket.SOCK_STREAM)    # AF_INET= IPv4  & SOCK_STREAM =  TCP
S.connect(('localhost', 9999))
# We changed S.connect((socket.gethostname(), 9999)) to S.connect(('localhost', 9999))

msgtoS = "Message to server from MT Client"; 
print("To Svr: ",msgtoS) ; send_to_server(msgtoS) ; 
Svr_msg = receive_from_server(0.1)  
if Svr_msg != "": 
    print("From Svr: ", Svr_msg)
else: print("No reply from Svr")

msgtoS = "REGISTER SP"; 
print("To Svr: ",msgtoS) ; send_to_server(msgtoS)  
Svr_msg = receive_from_server(0.1)  
if Svr_msg != "": 
    print("From Svr: ", Svr_msg)
else: print("No reply from Svr")

msgtoS = "INVITE Dummy2"; 
print("To Svr: ",msgtoS) ; send_to_server(msgtoS) ; 
Svr_msg = receive_from_server(0.1) 
if Svr_msg != "": 
    print("From Svr: ", Svr_msg)
else: print("No reply from Svr")
    
####### From here on is the Custom code for 5.3.###############################################





msgtoS =  "BSPARAMS 0.000005 0.05 0 3200"
print("To Svr: ",msgtoS) ; send_to_server(msgtoS)  
Svr_msg = receive_from_server(1)  
if Svr_msg != "": 
    print("From Svr: ", Svr_msg)
else: print("No reply from Svr")  

""""
msgtoS=bschoose("BSMSG Dummy2 I") 
# print("To Svr: ",msgtoS)
send_to_server(msgtoS)  
Svr_msg = receive_from_server(5)  
if Svr_msg != "": 
    print("From Svr: ", Svr_msg)
else: print("No reply from Svr") 
"""
####### Sending the information by odd/even
oddevenS = bschoose("BSMSG Dummy2 S")
dataindex = oddevenS.index(" S ")
dataindex = dataindex + 3
oddevenS  = oddevenS[dataindex:]
sendData  = oddevenS
## This works print(oddevenS)

oddevenLimiter = len(oddevenS) // 2
evendata = [0]*oddevenLimiter
odddata =  [0]*oddevenLimiter



# Send even data
i=0
msgfromS = ""
evendataString = ""
for x in range(0,(oddevenLimiter*2),2):
    evendata[i] = oddevenS[x]
    #Convert to a bit string
    evendataString += str(evendata[i])
    if x % 2 != 0:
        print("Incorrect Data Sent at value ", x)
    ++i

print("To Svr: Sending Even Pacjet Data")    
msgtoS = "BSMSG Dummy2 S " + evendataString;
# print("To Svr: EVEN DATA SENT")
send_to_server(msgtoS) 
wait = len(msgtoS)/100000.0
msgfromSeven = receive_from_server(wait)
if "BSMSGFROM" in msgfromS:
    print("From Svr: EVEN DATA BACK")
    
print("length needed original " + str(len(oddevenS)))
    
# Send ODD data
i=0
msgfromS = ""
odddataString = ""
for x in range(1,((oddevenLimiter*2)),2):
    odddata[i] = oddevenS[x]
    #Convert to a bit string
    odddataString += str(odddata[i])
    if x % 2 == 0:
        print("Incorrect Data Sent at value ", x)
    ++i


print("To Svr: Sending ODD Packet Data")    
msgtoS = "BSMSG Dummy2 S " + odddataString;
# print("To Svr: EVEN DATA SENT")
send_to_server(msgtoS) 
wait = len(msgtoS)/100000.0
msgfromSodd = receive_from_server(wait)
if "BSMSGFROM" in msgfromSodd:
    print("From Svr: ODD DATA BACK")
    

# Putting data back together
def bitarray_to_numpyint16array(data):
    NA=np.packbits(data)
    NA=NA.tobytes()
    NA=np.frombuffer(NA, dtype=np.int16)
    NA=NA.byteswap()
    return NA

recievedData = ""


dataindex = msgfromSeven.index(" S ")
dataindex = dataindex + 3
msgfromSeven  = msgfromSeven[dataindex:]

dataindex = msgfromSodd.index(" S ")
dataindex = dataindex + 3
msgfromSodd  = msgfromSodd[dataindex:]


# print(msgfromSodd[13:30])
xeven = 0
xodd = 0
oddevenLimiterReturn = len(msgfromSeven) * 2
for i in range((oddevenLimiterReturn)-1):
    if i % 2 == 0:
        recievedData += msgfromSeven[xeven]
        xeven = xeven + 1
    else:
        recievedData += msgfromSodd[xodd]
        xodd = xodd + 1
        
#print(recievedData[:400])
# print(sendData[:400])

# Listen to the compiled odd/even packets
bslisten("BSMSGFROM Dummy2 S " + recievedData)

        

        
        
 


    
    
    
    

time.sleep(0.1)
KB_msg = "" ; 
while KB_msg != "QUIT":
    # _thread.start_new_thread (receive_from_keyboard_MT , (KB_msg,) )  # try this later
    KB_msg = receive_from_keyboard()
    msgtoS=KB_msg[:]  # Makes a COPY of the string contents
    if KB_msg[0:5] =="BSMSG": msgtoS=bschoose(KB_msg)
    if KB_msg[0:5] =="LSMSG": msgtoS=lschoose(KB_msg)                   
    to_Print(msgtoS)
    send_to_server(msgtoS) 
    wait = len(msgtoS)/100000.0 
    if wait < 5.0: wait=5.0
    msgfromS = receive_from_server(wait)
    if msgfromS != "": 
        from_Print(msgfromS)
    else: print("No reply from Svr in ",wait," seconds")
    if msgfromS[0:9] =="BSMSGFROM": bslisten(msgfromS)
    if msgfromS[0:9] =="LSMSGFROM": lslisten(msgfromS)
# end of while clause


    


time.sleep(1.0) # Wait 1 s before closing the socket
S.close()  # close the socket
print("Client terminates")


To Svr:  Message to server from MT Client
From Svr:  Message not understood
To Svr:  REGISTER SP
From Svr:  Your userName SP is now registered with the server
To Svr:  INVITE Dummy2
From Svr:  INVITED Dummy2 and Dummy2 auto-accepted
To Svr:  BSPARAMS 0.000005 0.05 0 3200
From Svr:  BSPARAMS: beP=0.000005 framelossP=0.05 burstFac=0 frameSize=3200
Orig speech:


To Svr: Even Data
send-message length =  1408023
length needed original 2816016
To Svr: ODD Data
send-message length =  1408023
From Svr: ODD DATA BACK
length of BS:  2816016
Received speech:


KeyboardInterrupt: 